In [2]:
import subprocess
import os
import pandas as pd

In [3]:
def make_anvio_contig_db(taxa, anvioBasedb, genomeFilePath, genomeID, sh_cmd_db):
    """Generate contig databases for all genomes"""
    # Make the directory if it does not already exist
    if not os.path.isdir('{}{}'.format(anvioBasedb, taxa)):
        os.mkdir('{}{}'.format(anvioBasedb, taxa))
        os.mkdir('{}{}/fna'.format(anvioBasedb, taxa))
        os.mkdir('{}{}/contig_db'.format(anvioBasedb, taxa))
    # Create the contig database if it does not exist
    if not os.path.isfile('{}{}/contig_db/{}_contigs.db'.format(anvioBasedb, taxa, genomeID)):
        # Format the FASTA file
        anvio_reformat_fasta_cmd = 'anvi-script-reformat-fasta {} --simplify-names --output-file {}{}/fna/{}.fna --seq-type NT'.format(genomeFilePath, anvioBasedb, taxa, genomeID)
        # Create the contig database
        anvio_contigdb_cmd = 'anvi-gen-contigs-database -o {}{}/contig_db/{}_contigs.db -f {}{}/fna/{}.fna'.format(anvioBasedb, taxa, genomeID, anvioBasedb, taxa, genomeID)
        # Add the commands to the shell script
        sh_cmd_db += anvio_reformat_fasta_cmd + '\n'
        sh_cmd_db += anvio_contigdb_cmd + '\n'
        print (anvio_contigdb_cmd)
    return sh_cmd_db

def add_to_storage(external_genome_storage_file, taxa, genome, anvioBasedb):
    # Add the genome and its location to the file
    external_genome_storage_file += '{}\t{}{}/contig_db/{}_contigs.db\n'.format(genome, anvioBasedb, taxa, genome)
    # Return the updated file
    return external_genome_storage_file

def run_kegg_annot(sh_genome_kegg, taxa, genome, anvioBasedb):
    # First, add the command to the shell script
    sh_genome_kegg += 'anvi-run-kegg-kofams -T 64 -c {}{}/contig_db/{}_contigs.db\n'.format(anvioBasedb, taxa, genome)
    # Then return the shell script
    return sh_genome_kegg

def run_metabolism_reconstruction(sh_cmd_metabolism, taxa, genome, anvioBasedb):
    # First, add the command to the shell script
    sh_cmd_metabolism += 'anvi-estimate-metabolism -c {}{}/contig_db/{}_contigs.db -O {}{}/metabolism/{}-METABOLISM.txt\n'.format(anvioBasedb, taxa, genome, anvioBasedb, taxa, genome)
    # Then return the shell script
    return sh_cmd_metabolism

In [8]:
# This code takes in a dataframe of genomes and their taxonomic annotations, and creates a set of commands for running anvio to generate the contig databases, the genomes storage, and the pangenome for each genus.
# The output is a set of shell scripts that can be run on a cluster, with one script for each step.
# taxa_df is a dataframe with the genome names as the index and the first column containing the genus names and the second column containing the species names.
# fastaDir is the directory in which the fasta files are located.
# anvioBasedb is the directory in which the anvio databases are stored.

taxa_df = pd.read_csv('/DATA_RAID2/vtracann/shared/db/isolates/family_gtdbtk_annot.tsv', sep='\t', index_col=0, header=None)
anvioBasedb = '/DATA_RAID2/vtracann/shared/db/isolates/anvio/'
fastaDir = '/DATA_RAID2/vtracann/shared/db/isolates/global/unzipped/'
#print (taxa_df)
sh_cmd_db = ""
sh_cmd_storage = ""
sh_cmd_pangenome = ""
sh_genome_kegg = ""
sh_cmd_metabolism = ""

for taxa in set(taxa_df.loc[:, 1].values):
    if taxa != 'unclassified':
        genomes = taxa_df[taxa_df.loc[:, 1] == taxa]
        if taxa == 'Sphingomonadaceae':
            print (taxa)
            print (len(genomes))

        if len(genomes) > 10:
            external_genome_storage_file = "name\tcontigs_db_path\n"
            #print (taxa)
            #print (len(genomes))
            for genome in genomes.index:
                genomesFileName = [x for x in os.listdir(fastaDir) if x.startswith(genome)][0]
                genomeFilePath = fastaDir+genomesFileName
                sh_cmd_db = make_anvio_contig_db(taxa, anvioBasedb, genomeFilePath, genome, sh_cmd_db)
                external_genome_storage_file = add_to_storage(external_genome_storage_file, taxa, genome, anvioBasedb)
                if not os.path.isfile('{}{}/{}_annot-GENOMES.db'.format(anvioBasedb, taxa, taxa)):
                    sh_genome_kegg = run_kegg_annot(sh_genome_kegg, taxa, genome, anvioBasedb)
                if not os.path.isfile('{}{}/metabolism/{}_metabolism'.format(anvioBasedb, taxa, genome)):
                    sh_cmd_metabolism = run_metabolism_reconstruction(sh_cmd_metabolism, taxa, genome, anvioBasedb)
                    
            #print (len(os.listdir('{}{}/contig_db/'.format(anvioBasedb, taxa))))
                
            externaldbFile = open('{}{}/{}_external_db.tsv'.format(anvioBasedb, taxa, taxa), 'w')
            externaldbFile.write(external_genome_storage_file)
            externaldbFile.close()
            
            sh_cmd_storage+= "anvi-gen-genomes-storage -e {}{}/{}_external_db.tsv -o {}{}/{}-GENOMES.db\n".format(anvioBasedb, taxa, taxa, anvioBasedb, taxa, taxa)
            
            if not os.path.isdir('{}{}/pangenome/'.format(anvioBasedb, taxa)):
                os.mkdir('{}{}/pangenome/'.format(anvioBasedb, taxa))
            sh_cmd_pangenome+= "anvi-pan-genome -g {}{}/{}-GENOMES.db --mcl-inflation 1.4 --min-occurrence 2 -n {} -O {}{}/pangenome/ -T 64\n".format(anvioBasedb, taxa, taxa, taxa, anvioBasedb, taxa)
            
            if not os.path.isdir('{}{}/metabolism/'.format(anvioBasedb, taxa)):
                os.mkdir('{}{}/metabolism/'.format(anvioBasedb, taxa))
            
            

outFile = open('anvio_make_contigdb.sh', 'w')
outFile.write(sh_cmd_db)
outFile.close()

outFile = open('anvio_make_genome_storage.sh', 'w')
outFile.write(sh_cmd_storage)
outFile.close()

outFile = open('anvio_make_pangenome.sh', 'w')
outFile.write(sh_cmd_pangenome)
outFile.close()

outFile = open('anvio_run_kegg_annot.sh', 'w')
outFile.write(sh_genome_kegg)
outFile.close()

outFile = open('anvio_run_metabolism.sh', 'w')
outFile.write(sh_cmd_metabolism)
outFile.close()


Sphingomonadaceae
1
